![](../images/cs312.png)
***

# 12 Prepare : Reading 

## 1 - Objectives

- Learn about ThreeJS.
- Get "hello World" up and running in a browser.


## 2 - Topic

### 2.1 - OpenGL

> Open Graphics Library (OpenGL)[3][4] is a cross-language, cross-platform application programming interface (API) for rendering 2D and 3D vector graphics. The API is typically used to interact with a graphics processing unit (GPU), to achieve hardware-accelerated rendering.
> 
> Silicon Graphics Inc., (SGI) started developing OpenGL in 1991 and released it in January 1992;[5] applications use it extensively in the fields of computer-aided design (CAD), virtual reality, scientific visualization, information visualization, flight simulation, and video games. Since 2006 OpenGL has been managed by the non-profit technology consortium Khronos Group.


**Hello World in OpenGL**

```C
#include <GL/glut.h>

void displayMe(void)
{
    glClear(GL_COLOR_BUFFER_BIT);
    glBegin(GL_POLYGON);
    glVertex3f(0.0, 0.0, 0.0);
    glVertex3f(0.5, 0.0, 0.0);
    glVertex3f(0.5, 0.5, 0.0);
    glVertex3f(0.0, 0.5, 0.0);
    glEnd();
    glFlush();
}

int main(int argc, char** argv)
{
    glutInit(&argc, argv);
    glutInitDisplayMode(GLUT_SINGLE);
    glutInitWindowSize(300, 300);
    glutInitWindowPosition(100, 100);
    glutCreateWindow("Hello world :D");
    glutDisplayFunc(displayMe);
    glutMainLoop();
    return 0;
}
```

### 2.2 - WebGL


> WebGL (Web Graphics Library) is a JavaScript API for rendering interactive 2D and 3D graphics within any compatible web browser without the use of plug-ins.[2] WebGL is integrated completely into all the web standards of the browser, allowing GPU-accelerated usage of physics and image processing and effects as part of the web page canvas. WebGL elements can be mixed with other HTML elements and composited with other parts of the page or page background.[3] WebGL programs consist of control code written in JavaScript and shader code that is written in OpenGL Shading Language (GLSL), a language similar to C or C++, and is executed on a computer's graphics processing unit (GPU). WebGL is designed and maintained by the non-profit Khronos Group.[4]


**Hello World in WebGL**

```html
<html>

<head>
<title>WebGL Hello World</title>
<meta http-equiv="content-type" content="text/html; charset=ISO-8859-1">

<script type="text/javascript" src="glMatrix-0.9.5.min.js"></script>

<script id="shader-fs" type="x-shader/x-fragment">
    precision mediump float;
    void main(void) {
        gl_FragColor = vec4(0.9, 0.3, 0.6, 1.0);
    }
</script>

<script id="shader-vs" type="x-shader/x-vertex">
    attribute vec3 Position;
    uniform mat4 u_ModelView;
    uniform mat4 u_Persp;
    void main(void) {
        gl_Position = u_Persp * u_ModelView * vec4(Position, 1.0);
    }
</script>
</head>


<body>
    <canvas id="webgl_hello_world" width="800" height="600"></canvas>
</body>


<script type="text/javascript">
    var gl;
    function initGL(canvas) {
        try {
            gl = canvas.getContext("webgl");
            gl.viewportWidth = canvas.width;
            gl.viewportHeight = canvas.height;
        } catch (e) {
        }
        if (!gl) {
            alert("WebGL is not avaiable on your browser!");
        }
    }
    function getShader(gl, id) {
        var shaderScript = document.getElementById(id);
        if (!shaderScript) {
            return null;
        }
        var str = "";
        var k = shaderScript.firstChild;
        while (k) {
            if (k.nodeType == 3) {
                str += k.textContent;
            }
            k = k.nextSibling;
        }
        var shader;
        if (shaderScript.type == "x-shader/x-fragment") {
            shader = gl.createShader(gl.FRAGMENT_SHADER);
        } else if (shaderScript.type == "x-shader/x-vertex") {
            shader = gl.createShader(gl.VERTEX_SHADER);
        } else {
            return null;
        }
        gl.shaderSource(shader, str);
        gl.compileShader(shader);
        if (!gl.getShaderParameter(shader, gl.COMPILE_STATUS)) {
            alert(gl.getShaderInfoLog(shader));
            return null;
        }
        return shader;
    }
    var shader_prog;
    function initShaders() {
        var fragmentShader = getShader(gl, "shader-fs");
        var vertexShader = getShader(gl, "shader-vs");
        shader_prog = gl.createProgram();
        gl.attachShader(shader_prog, vertexShader);
        gl.attachShader(shader_prog, fragmentShader);
        gl.linkProgram(shader_prog);
        if (!gl.getProgramParameter(shader_prog, gl.LINK_STATUS)) {
            alert("Could not initialise shaders");
        }
        gl.useProgram(shader_prog);
        shader_prog.positionLocation = gl.getAttribLocation(shader_prog, "Position");
        gl.enableVertexAttribArray(shader_prog.positionLocation);
        shader_prog.u_PerspLocation = gl.getUniformLocation(shader_prog, "u_Persp");
        shader_prog.u_ModelViewLocation = gl.getUniformLocation(shader_prog, "u_ModelView");
    }
    var triangleVertexPositionBuffer;
    function initBuffers() {
        triangleVertexPositionBuffer = gl.createBuffer();
        gl.bindBuffer(gl.ARRAY_BUFFER, triangleVertexPositionBuffer);
        var vertices = [
             0.0,  1.0,  0.0,
            -1.0, -1.0,  0.0,
             1.0, -1.0,  0.0
        ];
        gl.bufferData(gl.ARRAY_BUFFER, new Float32Array(vertices), gl.STATIC_DRAW);
        triangleVertexPositionBuffer.itemSize = 3;
        triangleVertexPositionBuffer.numItems = 3;
    }
    var mvMatrix = mat4.create();
    var pMatrix = mat4.create();
    function drawScene() {
        gl.viewport(0, 0, gl.viewportWidth, gl.viewportHeight);
        gl.clear(gl.COLOR_BUFFER_BIT | gl.DEPTH_BUFFER_BIT);
        mat4.perspective(45, gl.viewportWidth / gl.viewportHeight, 0.1, 100.0, pMatrix);
        mat4.identity(mvMatrix);
        //Move our triangle
        mat4.translate(mvMatrix, [0.0, 0.0, -4.0]);
        //Pass triangle position to vertex shader
        gl.bindBuffer(gl.ARRAY_BUFFER, triangleVertexPositionBuffer);
        gl.vertexAttribPointer(shader_prog.positionLocation, triangleVertexPositionBuffer.itemSize, gl.FLOAT, false, 0, 0);
        //Pass model view projection matrix to vertex shader
        gl.uniformMatrix4fv(shader_prog.u_PerspLocation, false, pMatrix);
        gl.uniformMatrix4fv(shader_prog.u_ModelViewLocation, false, mvMatrix);
        //Draw our lovely triangle
        gl.drawArrays(gl.TRIANGLES, 0, triangleVertexPositionBuffer.numItems);
    }
    (function loadWebGL(){
        var canvas = document.getElementById("webgl_hello_world");
        initGL(canvas);
        initShaders();
        initBuffers();
        gl.clearColor(0.0, 0.0, 0.0, 1.0);
        gl.enable(gl.DEPTH_TEST);
        drawScene();
    })();
</script>

</html>
```

### 2.3 - ThreeJS


> Three.js is a cross-browser JavaScript library and Application Programming Interface (API) used to create and display animated 3D computer graphics in a web browser. Three.js uses WebGL. The source code is hosted in a repository on GitHub.
>
> Three.js allows the creation of Graphical Processing Unit (GPU)-accelerated 3D animations using the JavaScript language as part of a website without relying on proprietary browser plugins.[3][4] This is possible thanks to the advent of WebGL.[5]
> 
> High-level libraries such as Three.js or GLGE, SceneJS, PhiloGL or a number of other libraries make it possible to author complex 3D computer animations that display in the browser without the effort required for a traditional standalone application or a plugin.[6]


**Hello World in WebGL**

```js
// Create an empty scene
var scene = new THREE.Scene();

// Create a basic perspective camera
var camera = new THREE.PerspectiveCamera( 75, window.innerWidth/window.innerHeight, 0.1, 1000 );
camera.position.z = 4;

// Create a renderer with Antialiasing
var renderer = new THREE.WebGLRenderer({antialias:true});

// Configure renderer clear color
renderer.setClearColor("#000000");

// Configure renderer size
renderer.setSize( window.innerWidth, window.innerHeight );

// Append Renderer to DOM
document.body.appendChild( renderer.domElement );

// Add to scene

// Create a Cube Mesh with basic material
var geometry = new THREE.BoxGeometry( 1, 1, 1 );
var material = new THREE.MeshBasicMaterial( { color: "#433F81" } );
var cube = new THREE.Mesh( geometry, material );

// Add cube to Scene
scene.add( cube );

// Render Loop
var render = function () {
  requestAnimationFrame( render );

  cube.rotation.x += 0.01;
  cube.rotation.y += 0.01;

  // Render the scene
  renderer.render(scene, camera);
};

render();
```

***
## 3 - Reading

- Download the files "three.min.js" and "threejsBase.html" from GitHub to show a spinning cube.



Websites

- [Three JS webstie](https://threejs.org/)
- [ThreeJS Hello World](https://threejs.org/docs/#manual/introduction/Creating-a-scene)
- [WebGL](https://en.wikipedia.org/wiki/WebGL)
- [OpenGL](https://en.wikipedia.org/wiki/OpenGL)